In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import sklearn

In [2]:
n_1 = 34
a = 10
p_1 = a/n_1

n_2 = 16
b = 4
p_2 = b/n_2

pp = (p_1*n_1 + p_2*n_2)/(n_1+n_2)

z = (p_1 - p_2)/np.sqrt(pp*(1-pp)*(1/n_1+1/n_2))

In [3]:
(1 - sp.stats.norm.cdf(z ,loc = 0 ,scale = 1))

0.37293045872523534

In [4]:
p_1 - p_2 + sp.stats.norm.ppf(1-0.025)*np.sqrt(p_1*(1-p_1)/n_1+p_2*(1-p_2)/n_2)

0.3057930157736072

In [5]:
p_1 - p_2 - sp.stats.norm.ppf(1-0.025)*np.sqrt(p_1*(1-p_1)/n_1+p_2*(1-p_2)/n_2)

-0.2175577216559601

In [6]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = sp.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - sp.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return sp.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - sp.stats.norm.cdf(z_stat)

In [7]:

data_exp = np.array( [1 if i < 10 else 0 for i in range(34)] )
data_ctrl = np.array( [1 if i < 4 else 0 for i in range(16)] )

In [8]:

print('95%% confidence interval for a difference: [%.4f, %.4f]' % proportions_diff_confint_ind(data_exp, data_ctrl))

95% confidence interval for a difference: [-0.2176, 0.3058]


In [9]:

print('p-value: %.4f' % proportions_diff_z_test(proportions_diff_z_stat_ind(data_exp, data_ctrl), 'greater'))

p-value: 0.3729


In [10]:
banknots = pd.read_csv('banknots.txt',sep = '\t')
banknots

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [42]:
data_1 = banknots[['X1','X2','X3']]
data_2 = banknots[['X4','X5','X6']]
y = banknots[['real']]
train_1,test_1,y_train_1,y_test_1 = sklearn.model_selection.train_test_split(data_1.to_numpy(),y.to_numpy(),test_size=0.25,random_state=1)
train_2,test_2,y_train_2,y_test_2 = sklearn.model_selection.train_test_split(data_2.to_numpy(),y.to_numpy(),test_size=0.25,random_state=1)

In [43]:
from sklearn.linear_model import LogisticRegression
reg_1 = LogisticRegression().fit(train_1,y_train_1)
reg_2 = LogisticRegression().fit(train_2,y_train_2)

C:\anaconda\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
y_reg_1 = reg_1.predict(test_1)
y_reg_2 = reg_2.predict(test_2)

In [55]:
y_test_1.reshape((-1))+y_reg_1

array([2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 0, 1, 0, 2, 1, 1,
       2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 2,
       2, 1, 0, 1, 0, 0], dtype=int64)

In [56]:
error_1 = (y_test_1.reshape((-1))+y_reg_1)
error_2 = (y_test_2.reshape((-1))+y_reg_2)

In [61]:
unique, counts = numpy.unique(error_1, return_counts=True)
p_error_1 = counts[1]/len(y_reg_1)
unique, counts = numpy.unique(error_2, return_counts=True)
p_error_2 = counts[1]/len(y_reg_2)

In [62]:
p_error_1

0.2

In [63]:
p_error_2

0.02

In [71]:
f = ((((y_reg_1==y_test_1)==0)+((y_reg_2==y_test_2))==1)).sum()/2

In [72]:
g = (y_reg_1<y_reg_2).sum()

In [79]:
y_reg_1==y_test_1.reshape((-1))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False,  True, False,  True,  True,  True,  True,
       False,  True, False,  True,  True])

In [97]:
f = (((y_reg_1==y_test_1.reshape((-1)))==1)*(((y_reg_2==y_test_2.reshape((-1))))==0)).sum()
g = (((y_reg_1==y_test_1.reshape((-1)))==0)*(((y_reg_2==y_test_2.reshape((-1))))==1)).sum()

In [98]:
((y_reg_1==y_test_1.reshape((-1)))==0)

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False,  True,
       False, False,  True,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False,  True,  True, False,  True, False, False, False, False,
        True, False,  True, False, False])

In [99]:
((y_reg_2==y_test_1.reshape((-1)))==0)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False])

In [100]:
f

0

In [101]:
g

9

In [106]:
z = (f-g)/np.sqrt(f+g-((f-g)**2)/len(y_test_2))

In [107]:
z

-3.312945782245396

In [109]:
sp.stats.norm.cdf(z)*2

0.0009231887438696775

In [124]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = sp.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [127]:
df = pd.read_csv('banknots.txt', header = 0, sep = '\t')
df.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [128]:

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(banknots, banknots.real, test_size=50,train_size=150,random_state=1)
clf1 = LogisticRegression(random_state=0).fit(X_train[['X1','X2','X3']], y_train)
clf2 = LogisticRegression(random_state=0).fit(X_train[['X4','X5','X6']], y_train)
error_1 = np.abs(clf1.predict(X_test[['X1','X2','X3']]) - y_test)
error_2 = np.abs(clf2.predict(X_test[['X4','X5','X6']]) - y_test)

In [129]:
error_1.sum()

10

In [130]:
error_2.sum()

1

In [131]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_rel(error_1, error_2))

z_stat = proportions_diff_z_stat_rel(error_1, error_2)
p_value = proportions_diff_z_test(z_stat)
print("z_stat = ", z_stat)
print("p_value = ", np.round(p_value, 4))

95% confidence interval for a difference between proportions: [0.073511, 0.286489]
z_stat =  3.312945782245396
p_value =  0.0009


In [153]:
z = (541.4 - 525)/(100/np.sqrt(100))

In [154]:
z

1.6399999999999977

In [155]:
1 - sp.stats.norm().cdf(z)

0.05050258347410397

In [156]:
np.round(proportions_diff_z_test(z, 'greater'), 4)

0.0505

In [157]:
n = 200000
mu = 525
sigma = 100

n_course = 100
mu_course = 541.4

In [158]:
Z = (mu_course - mu)/(sigma / np.sqrt(n_course))
np.round(proportions_diff_z_test(Z, 'greater'), 4)

0.0505

In [146]:
Z

1.6399999999999977